# 🎬 Professional AI Video Generator - GPU Server
## Complete System with Coqui TTS + DreamShaper XL

### ⚠️ IMPORTANT INSTRUCTIONS:
1. **Enable GPU**: Runtime → Change runtime type → T4 GPU → Save
2. **Run Cell 1** → Installs Python 3.10 environment (runtime restarts)
3. **Run Cell 1 AGAIN** → Installs all dependencies
4. **Run Cell 2, 3, 4...** → Everything will work!

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📦 CELL 1: INSTALL ALL DEPENDENCIES
# ═══════════════════════════════════════════════════════════════════════════════
# ⚠️ RUN THIS CELL TWICE:
#    First run: Installs Python 3.10, restarts runtime
#    Second run (after restart): Installs all packages
# ═══════════════════════════════════════════════════════════════════════════════

import sys

print(f"📍 Current Python: {sys.version}\n")

if sys.version_info >= (3, 12):
    print("⚠️  Python 3.12 detected - Coqui TTS needs Python 3.10!")
    print("🔄 Installing Python 3.10 environment...\n")
    
    !pip install -q condacolab
    import condacolab
    condacolab.install()
    
    print("\n✅ Python 3.10 environment installed!")
    print("⚠️  Runtime will restart automatically...")
    print("⚠️  AFTER RESTART: Run this cell again to install packages!")
    
else:
    print(f"✅ Python {sys.version_info.major}.{sys.version_info.minor} - Compatible!\n")
    print("="*80)
    print("📦 INSTALLING ALL DEPENDENCIES")
    print("="*80 + "\n")
    
    # FFmpeg
    print("🎬 Installing FFmpeg...")
    !apt-get update -qq > /dev/null 2>&1
    !apt-get install -y -qq ffmpeg > /dev/null 2>&1
    !ffmpeg -version | head -n 1
    print("   ✅ FFmpeg installed!\n")
    
    # Core packages
    print("📦 Installing Flask and core packages...")
    !pip install -q --upgrade pip
    !pip install -q flask flask-cors pyngrok requests
    print("   ✅ Flask packages installed!\n")
    
    # PyTorch
    print("🔥 Installing PyTorch (GPU)...")
    !pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
    print("   ✅ PyTorch installed!\n")
    
    # Coqui TTS
    print("🎤 Installing Coqui TTS...")
    !pip install -q TTS soundfile numpy scipy
    print("   ✅ Coqui TTS installed!\n")
    
    # DreamShaper XL
    print("🎨 Installing DreamShaper XL...")
    !pip install -q diffusers transformers accelerate safetensors
    print("   ✅ DreamShaper XL installed!\n")
    
    # Image tools
    print("📸 Installing image tools...")
    !pip install -q pillow opencv-python-headless
    print("   ✅ Image tools installed!\n")
    
    print("="*80)
    print("✅ ALL DEPENDENCIES INSTALLED SUCCESSFULLY!")
    print("="*80)
    print("\n👉 Now run Cell 2 to setup GPU and imports!")
    print("="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 CELL 2: SETUP GPU & IMPORTS
# ═══════════════════════════════════════════════════════════════════════════════

import gc
import torch
import json
import subprocess
import base64
import time
import io
import re
import numpy as np
from pathlib import Path
from PIL import Image
from concurrent.futures import ThreadPoolExecutor, as_completed

from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
from threading import Thread

# GPU Detection
print("="*80)
print("🔍 GPU DETECTION")
print("="*80)

if torch.cuda.is_available():
    device = 'cuda'
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✅ GPU ENABLED: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
    print(f"🔥 CUDA Version: {torch.version.cuda}")
else:
    device = 'cpu'
    print("⚠️  WARNING: GPU NOT DETECTED")

print(f"\n🚀 Device: {device}")
print("="*80)

# Create output directory
output_dir = Path('/content/outputs')
output_dir.mkdir(exist_ok=True)

print(f"\n📁 Output directory: {output_dir}")
print("\n✅ Setup complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎨 CELL 3: LOAD IMAGE GENERATION MODEL (DreamShaper XL)
# ═══════════════════════════════════════════════════════════════════════════════

from diffusers import StableDiffusionXLPipeline

# Global model variable
img_pipeline = None

# Style definitions for all 12 styles
STYLE_DEFINITIONS = {
    "cinematic": {
        "positive": "cinematic film still, movie quality, professional cinematography, depth of field, bokeh, film grain, dramatic lighting",
        "negative": "amateur, low quality, blurry, cartoon, anime"
    },
    "documentary": {
        "positive": "documentary photography, National Geographic style, photojournalism, natural lighting, authentic, real life",
        "negative": "staged, fake, cartoon, anime, low quality"
    },
    "anime": {
        "positive": "anime style, manga illustration, cel shaded, vibrant colors, Japanese animation, studio ghibli quality",
        "negative": "photorealistic, 3D render, western cartoon, low quality"
    },
    "horror": {
        "positive": "horror atmosphere, dark and creepy, ominous lighting, terrifying, nightmare fuel, psychological horror, eerie",
        "negative": "bright, cheerful, colorful, cute, low quality"
    },
    "comic": {
        "positive": "comic book art, graphic novel style, bold lines, dynamic composition, pop art colors, marvel comics style",
        "negative": "photorealistic, blurry, low quality, amateur"
    },
    "historical": {
        "positive": "historical photograph, vintage photo, sepia tone, aged photograph, 1920s style, grainy, archival quality",
        "negative": "modern, digital, colorful, cartoon, low quality"
    },
    "scifi": {
        "positive": "sci-fi concept art, cyberpunk, neon lights, futuristic technology, blade runner style, advanced technology",
        "negative": "historical, vintage, low quality, blurry"
    },
    "noir": {
        "positive": "film noir, high contrast black and white, dramatic shadows, 1940s detective aesthetic, cinematic",
        "negative": "colorful, bright, cartoon, low quality"
    },
    "fantasy": {
        "positive": "epic fantasy art, magical atmosphere, dungeons and dragons style, concept art, detailed, enchanted",
        "negative": "modern, realistic, photograph, low quality"
    },
    "3d": {
        "positive": "3D render, Pixar style, CGI graphics, Blender render, octane render, photorealistic 3D",
        "negative": "2D, flat, painting, sketch, low quality"
    },
    "sketch": {
        "positive": "pencil sketch, hand-drawn, charcoal drawing, line art, traditional art, artistic sketch",
        "negative": "photorealistic, 3D render, colorful, low quality"
    },
    "watercolor": {
        "positive": "watercolor painting, soft edges, flowing colors, artistic, traditional painting, hand-painted",
        "negative": "photorealistic, 3D, digital art, low quality"
    }
}

def load_image_model():
    """Load DreamShaper XL model"""
    global img_pipeline
    
    if img_pipeline is None:
        print("🎨 Loading DreamShaper XL (Supports ALL 12 styles!)...")
        
        img_pipeline = StableDiffusionXLPipeline.from_pretrained(
            "Lykon/dreamshaper-xl-1-0",
            torch_dtype=torch.float16,
            variant="fp16",
            use_safetensors=True
        ).to(device)
        
        img_pipeline.enable_xformers_memory_efficient_attention()
        
        print("   ✅ DreamShaper XL loaded!")
        print("   🎯 Supports: cinematic, documentary, anime, horror, comic, etc.")
        print("   📐 Resolution: 1536x864 (16:9 ratio)")
    
    return img_pipeline

def generate_single_image(prompt, style="cinematic"):
    """Generate single image with specific style"""
    pipeline = load_image_model()
    
    # Get style keywords
    style_def = STYLE_DEFINITIONS.get(style.lower(), STYLE_DEFINITIONS["cinematic"])
    
    # Build full prompt
    full_prompt = f"{prompt}, {style_def['positive']}, highly detailed, professional quality, 8k"
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Generate
    with torch.inference_mode():
        image = pipeline(
            prompt=full_prompt,
            negative_prompt=style_def['negative'],
            num_inference_steps=25,
            guidance_scale=7.5,
            height=864,
            width=1536
        ).images[0]
    
    return image

print("✅ Image generation functions loaded!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎤 CELL 4: VOICE MAPPING & TTS (Coqui TTS - VCTK Speakers)
# ═══════════════════════════════════════════════════════════════════════════════

from TTS.api import TTS
import soundfile as sf

# Global TTS model
tts_model = None

# Voice mapping
VOICE_MAPPING = {
    # Male voices
    'guy': 'p226', 'adam': 'p226', 'brian': 'p227', 'andrew': 'p243',
    'michael': 'p232', 'george': 'p254', 'christopher': 'p259', 'davis_deep': 'p273',
    # Female voices
    'aria': 'p229', 'sarah': 'p231', 'nicole': 'p233', 'jenny': 'p228',
    'emma': 'p230', 'emma_british': 'p236', 'isabella': 'p244', 'sara': 'p231',
    # Defaults
    'default_male': 'p226', 'default_female': 'p229',
}

def get_coqui_speaker(voice_id):
    """Get Coqui VCTK speaker ID from frontend voice ID"""
    return VOICE_MAPPING.get(voice_id, 'p226')

def load_tts_model():
    """Load Coqui TTS model"""
    global tts_model
    
    if tts_model is None:
        print("🎤 Loading Coqui TTS (PyTorch GPU)...")
        tts_model = TTS("tts_models/en/vctk/vits", progress_bar=False).to(device)
        print("   ✅ Coqui TTS loaded!")
    
    return tts_model

def split_text_smart(text, max_chars=1000):
    """Split text into optimized chunks"""
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_chars:
            current_chunk += sentence + " "
        else:
            if current_chunk:
                chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    
    if current_chunk:
        chunks.append(current_chunk.strip())
    
    return chunks if chunks else [text]

def generate_audio_parallel(text, voice_id, speed=1.0):
    """Generate audio with parallel processing"""
    tts = load_tts_model()
    speaker = get_coqui_speaker(voice_id)
    
    chunks = split_text_smart(text, max_chars=1000)
    
    if len(chunks) == 1:
        # Single chunk
        temp_file = io.BytesIO()
        tts.tts_to_file(text=text, speaker=speaker, file_path=temp_file)
        temp_file.seek(0)
        audio, sample_rate = sf.read(temp_file)
        return audio, sample_rate
    
    # Parallel processing
    def generate_chunk(chunk_text, idx):
        temp_file = io.BytesIO()
        tts.tts_to_file(text=chunk_text, speaker=speaker, file_path=temp_file)
        temp_file.seek(0)
        audio, sr = sf.read(temp_file)
        return idx, audio, sr
    
    all_audio = []
    sample_rate = None
    
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = {executor.submit(generate_chunk, chunk, i): i for i, chunk in enumerate(chunks)}
        for future in as_completed(futures):
            idx, audio, sr = future.result()
            all_audio.append((idx, audio))
            sample_rate = sr
    
    all_audio.sort(key=lambda x: x[0])
    combined = np.concatenate([audio for _, audio in all_audio])
    
    if speed != 1.0:
        from scipy import signal
        samples = int(len(combined) / speed)
        combined = signal.resample(combined, samples)
    
    return combined, sample_rate

print("="*80)
print("🎤 VOICE SYSTEM CONFIGURED (COQUI TTS)")
print("="*80)
print(f"✅ Total voices: {len(VOICE_MAPPING)}")
print("="*80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎬 CELL 5: VIDEO COMPILATION (FFmpeg)
# ═══════════════════════════════════════════════════════════════════════════════

def compile_video_mixed_media(media_data, media_types, audio_data, durations, effects, captions=None):
    """Compile video with unlimited media items + captions"""
    print(f"🎬 Compiling video with {len(media_data)} items...")
    
    temp_dir = output_dir / "temp"
    temp_dir.mkdir(exist_ok=True)
    
    # Save media files
    media_paths = []
    for i, (data, media_type) in enumerate(zip(media_data, media_types)):
        ext = 'png' if media_type == 'image' else 'mp4'
        media_path = temp_dir / f"media_{i:03d}.{ext}"
        
        if isinstance(data, str) and data.startswith('data:'):
            data = data.split(',')[1]
        
        media_bytes = base64.b64decode(data)
        with open(media_path, 'wb') as f:
            f.write(media_bytes)
        media_paths.append(media_path)
    
    # Save audio
    audio_path = temp_dir / "audio.wav"
    if isinstance(audio_data, str) and audio_data.startswith('data:'):
        audio_data = audio_data.split(',')[1]
    audio_bytes = base64.b64decode(audio_data)
    with open(audio_path, 'wb') as f:
        f.write(audio_bytes)
    
    # Process media with effects
    processed_paths = []
    for i, (media_path, media_type, duration) in enumerate(zip(media_paths, media_types, durations)):
        processed_path = temp_dir / f"processed_{i:03d}.mp4"
        
        filters = ["scale=1920:1080:force_original_aspect_ratio=decrease,pad=1920:1080:(ow-iw)/2:(oh-ih)/2"]
        
        if effects.get('zoom_effect', True):
            filters.append("zoompan=z='min(zoom+0.0015,1.1)':d=1:x=iw/2-(iw/zoom/2):y=ih/2-(ih/zoom/2):s=1920x1080")
        
        if effects.get('color_filter') == 'cinematic':
            filters.append("eq=contrast=1.1:saturation=0.9")
        
        if media_type == 'image':
            filters.append("fps=24")
        
        video_filter = ','.join(filters)
        
        if media_type == 'image':
            cmd = ['ffmpeg', '-y', '-loop', '1', '-i', str(media_path), '-t', str(duration),
                   '-vf', video_filter, '-c:v', 'libx264', '-preset', 'ultrafast',
                   '-pix_fmt', 'yuv420p', str(processed_path)]
        else:
            cmd = ['ffmpeg', '-y', '-i', str(media_path), '-t', str(duration),
                   '-vf', video_filter, '-c:v', 'libx264', '-preset', 'ultrafast',
                   '-c:a', 'copy', str(processed_path)]
        
        subprocess.run(cmd, capture_output=True)
        processed_paths.append(processed_path)
    
    # Concatenate clips
    concat_file = temp_dir / "concat.txt"
    with open(concat_file, 'w') as f:
        for path in processed_paths:
            f.write(f"file '{path}'\n")
    
    temp_video_path = temp_dir / "temp_video.mp4"
    subprocess.run(['ffmpeg', '-y', '-f', 'concat', '-safe', '0', '-i', str(concat_file),
                    '-c:v', 'copy', str(temp_video_path)], capture_output=True)
    
    # Add audio
    output_file = output_dir / "final_video.mp4"
    subprocess.run(['ffmpeg', '-y', '-i', str(temp_video_path), '-i', str(audio_path),
                    '-c:v', 'copy', '-c:a', 'aac', '-b:a', '192k',
                    '-shortest', str(output_file)], capture_output=True)
    
    # Cleanup
    for path in media_paths + processed_paths + [concat_file, audio_path, temp_video_path]:
        if Path(path).exists():
            Path(path).unlink()
    
    print(f"   ✅ Video compiled: {output_file.name}")
    return output_file

print("✅ Video compilation functions loaded!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🚀 CELL 6: FLASK SERVER & ALL ROUTES
# ═══════════════════════════════════════════════════════════════════════════════

app = Flask(__name__)
CORS(app)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy',
        'device': device,
        'gpu': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None',
        'features': {'voices': len(VOICE_MAPPING), 'coqui_tts': True, 'dreamshaper_xl': True}
    })

@app.route('/generate_audio', methods=['POST'])
def generate_audio():
    """Generate audio from text"""
    try:
        data = request.json
        text = data.get('text', '')
        voice = data.get('voice', 'aria')
        speed = float(data.get('speed', 1.0))
        
        if not text:
            return jsonify({'error': 'No text provided'}), 400
        
        final_audio, sample_rate = generate_audio_parallel(text, voice, speed)
        
        audio_path = output_dir / f"audio_{hash(text)}.wav"
        sf.write(str(audio_path), final_audio, sample_rate)
        
        return send_file(audio_path, mimetype='audio/wav', as_attachment=True)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/generate_image', methods=['POST'])
def generate_image():
    """Generate single image"""
    try:
        data = request.json
        prompt = data.get('prompt', '')
        style = data.get('style', 'cinematic')
        
        if not prompt:
            return jsonify({'error': 'No prompt provided'}), 400
        
        image = generate_single_image(prompt, style)
        
        image_path = output_dir / f"image_{hash(prompt)}.png"
        image.save(image_path, format='PNG')
        
        return send_file(image_path, mimetype='image/png', as_attachment=True)
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/generate_images_batch', methods=['POST'])
def generate_images_batch():
    """Generate multiple images in batch"""
    try:
        data = request.json
        scenes = data.get('scenes', [])
        style = data.get('style', 'cinematic')
        
        if not scenes:
            return jsonify({'error': 'No scenes provided'}), 400
        
        results = []
        for i, scene in enumerate(scenes):
            prompt = scene.get('description', '')
            if not prompt:
                results.append({'success': False, 'error': 'No prompt', 'scene_index': i})
                continue
            
            try:
                image = generate_single_image(prompt, style)
                
                buffer = io.BytesIO()
                image.save(buffer, format='PNG')
                image_bytes = buffer.getvalue()
                image_base64 = base64.b64encode(image_bytes).decode('utf-8')
                
                results.append({
                    'success': True,
                    'image_data': image_base64,
                    'scene_index': i,
                    'resolution': '1536x864',
                    'model': 'DreamShaper-XL',
                    'style': style
                })
            except Exception as e:
                results.append({'success': False, 'error': str(e), 'scene_index': i})
        
        return jsonify({'results': results})
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/compile_video', methods=['POST'])
def compile_video():
    """Compile video from media and audio"""
    try:
        data = request.json
        media_data = data.get('media', [])
        media_types = data.get('media_types', [])
        audio_data = data.get('audio', '')
        durations = data.get('durations', [])
        effects = data.get('effects', {})
        captions = data.get('captions')
        
        if not media_data or not audio_data:
            return jsonify({'error': 'Media and audio required'}), 400
        
        if not media_types:
            media_types = ['image'] * len(media_data)
        
        video_path = compile_video_mixed_media(media_data, media_types, audio_data, durations, effects, captions)
        
        return send_file(video_path, mimetype='video/mp4', as_attachment=True, download_name='final_video.mp4')
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/generate_complete_video', methods=['POST'])
def generate_complete_video():
    """Generate complete video with images, voice, and effects"""
    try:
        data = request.json
        script = data.get('script', '')
        image_prompts = data.get('image_prompts', [])
        voice_id = data.get('voice_id', 'aria')
        effects = data.get('effects', {})
        captions = data.get('captions', [])
        durations = data.get('durations', [])
        style = data.get('style', 'cinematic')
        speed = float(data.get('speed', 1.0))
        
        if not script or not image_prompts:
            return jsonify({'error': 'Script and prompts required'}), 400
        
        print(f"\n📝 Generating complete video:")
        print(f"   Script: {len(script)} chars")
        print(f"   Images: {len(image_prompts)} scenes")
        print(f"   Voice: {voice_id}, Style: {style}")
        
        # Generate images
        print(f"\n🎨 Generating {len(image_prompts)} images...")
        image_paths = []
        for i, prompt in enumerate(image_prompts, 1):
            print(f"   [{i}/{len(image_prompts)}] {prompt[:50]}...")
            image = generate_single_image(prompt, style)
            image_path = output_dir / f"scene_{i:03d}.png"
            image.save(image_path, format='PNG')
            image_paths.append(image_path)
        
        # Generate voice
        print(f"\n🎤 Generating voice narration...")
        final_audio, sample_rate = generate_audio_parallel(script, voice_id, speed)
        audio_path = output_dir / "narration.wav"
        sf.write(str(audio_path), final_audio, sample_rate)
        audio_duration = len(final_audio) / sample_rate
        
        # Prepare media
        media_data = []
        media_types = []
        for img_path in image_paths:
            with open(img_path, 'rb') as f:
                img_bytes = f.read()
                img_base64 = base64.b64encode(img_bytes).decode('utf-8')
                media_data.append(img_base64)
                media_types.append('image')
        
        with open(audio_path, 'rb') as f:
            audio_bytes = f.read()
            audio_base64 = base64.b64encode(audio_bytes).decode('utf-8')
        
        if not durations or len(durations) != len(image_prompts):
            duration_per_image = audio_duration / len(image_prompts)
            durations = [duration_per_image] * len(image_prompts)
        
        # Compile video
        video_path = compile_video_mixed_media(media_data, media_types, audio_base64, durations, effects, captions if captions else None)
        
        # Cleanup
        for img_path in image_paths:
            if img_path.exists():
                img_path.unlink()
        if audio_path.exists():
            audio_path.unlink()
        
        print(f"\n✅ Complete video generated!")
        
        return send_file(video_path, mimetype='video/mp4', as_attachment=True, download_name='final_video.mp4')
    except Exception as e:
        import traceback
        traceback.print_exc()
        return jsonify({'error': str(e)}), 500

print("✅ Flask server configured with all routes!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌐 CELL 7: START SERVER WITH NGROK
# ═══════════════════════════════════════════════════════════════════════════════

# Configure ngrok
NGROK_AUTH_TOKEN = "2gacBVJPhjmZXiON9TYHmhI8TkN_39DL8HL8ug3xR7i6QWw9h"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Start Flask server in background
def run_server():
    app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(3)

# Start ngrok tunnel
public_url = ngrok.connect(5001, bind_tls=True)

print("\n" + "="*80)
print("🎉 GPU SERVER RUNNING - COQUI TTS + DREAMSHAPER XL!")
print("="*80)
print(f"\n📡 Public URL: {public_url.public_url}")
print(f"🖥️  Local URL:  http://localhost:5001")

if torch.cuda.is_available():
    print(f"\n🔥 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

print("\n📌 API Endpoints:")
print(f"   {public_url.public_url}/health")
print(f"   {public_url.public_url}/generate_audio")
print(f"   {public_url.public_url}/generate_image")
print(f"   {public_url.public_url}/generate_images_batch")
print(f"   {public_url.public_url}/compile_video")
print(f"   {public_url.public_url}/generate_complete_video")

print("\n" + "="*80)
print("🔧 UPDATE YOUR BACKEND CONFIG:")
print("="*80)
print(f"   File: story-video-generator/config/__init__.py")
print(f"   Set: COLAB_SERVER_URL = '{public_url.public_url}'")

print("\n" + "="*80)
print("✅ ALL FEATURES READY:")
print("="*80)
print("   • Coqui TTS (13 voices, PyTorch GPU)")
print("   • DreamShaper XL (12 styles: cinematic, anime, horror, etc.)")
print("   • FFmpeg video compilation with effects")
print("   • Parallel audio processing (4x faster)")
print("="*80)
print("\n🌟 Server ready! Copy the URL to your config file.")
print("\nPress Ctrl+C to stop.\n")

# Keep server running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n🛑 Server stopped!")